In [ ]:
%pylab inline


import matplotlib.pyplot as plt
import numpy as np

import cartopy.crs as ccrs
from cartopy.io.srtm import srtm_composite

from osgeo import gdal
from osgeo import gdal_array

## Google map tiles with cartopy !

For a given lat / lon box, we can download (on demand) map tiles and plot them ... 


In [ ]:
from cartopy.io.img_tiles import GoogleTiles

# Specify a region of interest

lat0 =  33  ; lat1 = 37
lon0 =  -120; lon1 = -116

gg_tiles = GoogleTiles()

plt.figure(figsize=(10, 10))


ax = plt.subplot(111, projection=gg_tiles.crs)
ax.set_extent([lon0, lon1, lat0, lat1])
ax.add_image(gg_tiles, 11)

plt.scatter(lon0, lat0, marker=(5, 1), color='red', s=200)
gl = ax.gridlines(draw_labels=True,)
gl.xlabels_top = False
gl.ylabels_left = False

In [ ]:
# plt.figure(figsize=(15, 10))
# ax = plt.subplot(111, projection=ccrs.PlateCarree())

# elev, crs, extent = srtm_composite(12, 47, 1, 1)
# plt.imshow(elev, extent=extent, transform=crs,
#            cmap='gist_earth', origin='lower')
# cb = plt.colorbar(orientation='vertical')
# cb.set_label('Altitude')
# plt.title("SRTM Map")
# gl = ax.gridlines(draw_labels=True,)
# gl.xlabels_top = False
# gl.ylabels_left = False


plt.figure(figsize=(15, 10))
ax = plt.subplot(111, projection=ccrs.PlateCarree())

elev, crs, extent = srtm_composite(lon0, lat0, 1, 1)
elev = np.ma.masked_less_equal(elev,0,copy=False)
plt.imshow(elev, extent=extent, transform=crs,
           cmap='gist_earth', origin='lower')
cb = plt.colorbar(orientation='vertical')
cb.set_label('Altitude')
plt.title("SRTM Map")
gl = ax.gridlines(draw_labels=True,)
gl.xlabels_top = False
gl.ylabels_left = False



In [ ]:
plt.figure(figsize=(15, 10))
ax = plt.subplot(111, projection=ccrs.PlateCarree())

elev, crs, extent = srtm_composite(-120, 33, 4, 4)
elev = np.ma.masked_less_equal(elev, -100, copy=False)

plt.imshow(elev, extent=extent, transform=crs,
           cmap='terrain', origin='lower')
cb = plt.colorbar(orientation='vertical')
cb.set_label('Altitude')
plt.title("SRTM Map")
gl = ax.gridlines(draw_labels=True,)
gl.xlabels_top = False
gl.ylabels_left = False

In [ ]:
# Patching holes in the data by a smoothing / interpolation routine from gdal

elev, crs, extent = srtm_composite(-120, 33, 4, 4)
src_ds = gdal_array.OpenArray(elev)
srcband = src_ds.GetRasterBand(1)
dstband = srcband
maskband = srcband
smoothing_iterations = 0
options = []
max_distance = 0.1
result = gdal.FillNodata(dstband, maskband,
                         max_distance, smoothing_iterations, options,
                         callback=None)
elev = dstband.ReadAsArray()



In [ ]:
# And plotting

plt.figure(figsize=(15, 10))
ax = plt.subplot(111, projection=ccrs.PlateCarree())

plt.imshow(elev, extent=extent, transform=crs, cmap='terrain', origin='lower', vmin=0.0, vmax=4000.0)

cb = plt.colorbar(orientation='vertical')
cb.set_label('Altitude')
plt.title("SRTM Map")
gl = ax.gridlines(draw_labels=True,)
gl.xlabels_top = False
gl.ylabels_left = False